<a href="https://www.kaggle.com/code/robertomoragadiaz/notebook-data-analytics-solucion?scriptVersionId=193444900" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## INICIO
escribe aquí lo que quieras.

*esto es Markdown*

## Intro

**Análisis de datos:** es el proceso de examinar, limpiar, transformar e interpretar datos para extraer información útil, apoyar la toma de decisiones y generar conocimiento.

**Importancia:**
Es crucial porque permite a las organizaciones y personas tomar decisiones informadas basadas en hechos y patrones (data-driven) para: 
* optimizar procesos, 
* identificar oportunidades y riesgos, y 
* mejorar resultados en diversos ámbitos, desde negocios hasta ciencia y tecnología.


El análisis de datos es fundamental para la experimentación y innovación porque proporciona una base objetiva para evaluar hipótesis, medir resultados y validar nuevas ideas. 

## Carga de librerias

In [ ]:
#Importamos librerias de la siguiente forma:
import numpy as np # importando numpy
from scipy import stats # importando scipy.stats
import matplotlib.pyplot as plt #importando matplotlib
import pandas as pd #pandas
import seaborn as sns

## Soltar las manos

In [ ]:
#crea un dataset con la vida media de los clientes del banco
vida = np.random.uniform(3, 32, size=100)

print(vida)

In [ ]:
#calcula la media con 1 decimal
media_vida = np.mean(vida).round(1)
print(media_vida, 'meses')

In [ ]:
#crea dataset con el ARPU del cliente
media= 22 #dolares
sigma= 3 #dolares

arpu = np.random.normal(media, sigma, size=100)

print(arpu)

In [ ]:
#graficar ARPU
plt.hist(arpu)
plt.show()

In [ ]:
#Seaborn
sns.scatterplot(x=vida, y=arpu)
plt.show()

## Preparar la base de datos

In [ ]:
#Cargamos la base de datos a estudiar
data = pd.read_csv("/kaggle/input/bbdd-kanban-game-01/BBDD_kanban_game_kaggle - Hoja 1.csv")

#Muestra los primeros registros para verificar que se ha cargado correctamente
data.head()

#prueba con: data.tail()

vemos datos NaN, debemos limpiar la base

Nota: otra forma de leer un dataset:

> #base de datos en google sheet compartida como CSV

> url = 'https://docs.google.com/spreadsheets/d/1QY3gsmqj8ulg0dHGvkumH2rLMUy039bEBPL43Hg9gMo/pub?gid=0&single=true&output=csv'

> #lectura de la 'base de datos' de los equipos agile

> df = pd.read_csv(url)

In [ ]:
#quitar registros sin leadtime
base = data.dropna(subset=['Lead time'])

#reemplazar registros con NAN por cero (0)
base = base.fillna(0)

base.head()

In [ ]:
#ver los tipos de datos
print(base.dtypes)

In [ ]:
#Una vista rápida de los campos númericos y sus estadísticos principales

base.describe()

Vamos a realizar una consulta para conocer los Equipos que contiene la base y la cantidad de experimentos que ha ejecutado cada equipo.

In [ ]:
#crear tabla pivote
table1 = pd.pivot_table(
    data=base,
    index=['Equipo'],
    values=['Ciclo/experimento'], 
    aggfunc={'Ciclo/experimento': 'max'}, 
    margins=False
)
# Mostrar la tabla pivote
table1

In [ ]:
#Otra forma
# Crear tabla pivote con conteo de valores únicos en 'Ciclo/experimento' por 'Equipo'
table1 = pd.pivot_table(
    data=base,
    index=['Equipo'],
    values=['Ciclo/experimento'],
    aggfunc={'Ciclo/experimento': 'nunique'},
    margins=False
)

# Mostrar la tabla pivote
table1

## Filtrar la muestra de trabajo

Queremos trabajar solo con el dataset de los experimentos del equipo 'Team ML'

In [ ]:
#Filtrando experimentos solo del equipo ML

ml = base.query('Equipo == "Team ML"')
ml.head()


#otro filtro si quisieramos mirar registros de equipo ML y con WIP mayor o igual a 6.
#ml = base.query('Equipo == "Team ML" and WIP >= 6')

In [ ]:
#Creamos una vista general, para conocer por cada equipo los resultados de:
#cantidad de productos completados (cantidad), leadtime (media), tasa de defectos (media) y utilizacion (max)
#esto por el nivel de WIP exhibido en cada escenario de trabajo (3, 6,  y  push (20)

table1 = pd.pivot_table(
    data=ml,
    index=['Equipo', 'WIP'],
    values=['Producto terminado','Lead time','utilizacion', 'q Defecto', 'Q semiterminado', 'Valor'], 
    aggfunc={'Producto terminado':(np.size), 'Lead time': "mean", 'utilizacion':"max",'q Defecto':"sum", 'Q semiterminado':'sum', 'Valor':"sum"}, 
    margins=False #probar con True y ver qué ocurre
)
table1

## Análisis de datos y visualizaciones

### Crear histogramas

#### Con pandas

Ojo: pandas sigue utilizando matplotlib en segundo plano para la visualización.

In [ ]:
# Crear el histograma para la columna 'Lead time'
ml['Lead time'].plot(kind='hist', bins=10, color='blue', edgecolor='black')

# Configurar los títulos y etiquetas
plt.title('Histograma de Lead Time')
plt.xlabel('Lead Time')
plt.ylabel('Frecuencia')
plt.show()

In [ ]:
#Calcular la media, mediana y desviación estandar

media = np.mean(ml['Lead time'])
mediana = np.median(ml['Lead time'])
sigma = np.std(ml['Lead time'])


print('Media: ', media)
print('Median: ', mediana)
print('Desv. std: ', sigma.round(1)) #para redondear el valor


### Con Matplotlib

### Un ejemplo práctico del uso de matplotlib (mostrando sus potencialidades)
La imagen de un agujero negro captado por múltiples telescopios y procesado con matplotlib [(Enlace al repositorio de código en github)](https://github.com/achael/eht-imaging):


![Agujero Negro](https://www.muylinux.com/wp-content/uploads/2019/04/bh.jpg)


Esto fue posible gracias a la investigación de la Dr. Katie Boumann y el equipo involucrado en el desarrollo de la librería [(Charla Ted relacionada)](https://www.youtube.com/watch?v=BIvezCVcsYs)

<img src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRfacBn6MpmkpOr9JeywRqK_KBBJ553BdV24jz7y9L8qXQYNI-nY4OtgmoJWBWFSQxR9sM&usqp=CAU">

In [ ]:
# Crear el histograma para la columna 'Lead time'
plt.hist(ml['Lead time'], bins=10, color='blue', edgecolor='black', range=[0, 200], density=True, alpha=0.2)

# Configurar los títulos y etiquetas
plt.title('Histograma de Lead Time')
plt.xlabel('Lead Time')
plt.ylabel('Frecuencia')

# Añadir una línea horizontal en el eje X en 90 unidades de tiempo
plt.axvline(90, color='springgreen', linestyle='--', label='Target \nService time')

# Mostrar el histograma
plt.legend()
plt.show()


Histograma de leadtime segmentado por nivel de WIP

In [ ]:
# Filtrar el dataset para WIP = 20
ml_wip20 = ml.query('WIP == 20')
ml_wip6 = ml.query('WIP == 6')
ml_wip3 = ml.query('WIP == 3')

# Crear el runchart
plt.figure(figsize=(10, 6))
plt.hist(ml_wip20['Lead time'], bins=10, range=(10, 200), color='r', alpha=0.2, label='WIP=20', density=True)
plt.hist(ml_wip6['Lead time'], bins=10, range=(10, 200), color='b', alpha=0.2, label='WIP=6', density=True)
plt.hist(ml_wip3['Lead time'], bins=10, range=(10, 200), color='g', alpha=0.1, label='WIP=3', density=True)
plt.xlabel('Lead time')
plt.ylabel('Frecuencia')
plt.title('Histograma de leadtime para distintos escenarios de WIP Limit')
plt.legend()
plt.show()

Al parecer si segmentamos por nivel de WIP los leadtime son diferentes.

**Preguntas (algunas) que surjen del análisis exploratorio de datos:**
¿Cuál nivel de WIP ofrece mejores tiempos 'Lead time' ?
¿existe una relación entre el nivel de WIP y el Leadtime ?

### Crear una gráfica de dispersión

* existe relación entre el tiempo 'leadtime' de los productos completados y la cantidad de WIP?
* existe relación entre el tiempo 'leadtime' y la calidad del producto?


In [ ]:
# Crear un grafico de dispersion
plt.figure(figsize=(10, 6))

sns.scatterplot(ml, x='WIP', y='Lead time', color='r')
# Definir los rangos de los ejes X e Y
#plt.ylim([0, 1])  # Reemplaza 'ymin' y 'ymax' con los valores deseados

plt.xlabel('Nivel de WIP (Work in Progress)')
plt.ylabel('Lead time')
plt.title('Relación entre el WIP y el Lead time')
plt.show()

La gráfica anterior nos arroja que existe una relación positiva:

a mayor WIP , existen mayores Lead time.

*Nota: recomendamos investigar sobre Ley de Little.*

In [ ]:
# Crear un grafico de dispersion
plt.figure(figsize=(10, 6))

sns.scatterplot(ml, x='Lead time', y='q Defecto', color='r')
# Definir los rangos de los ejes X e Y
#plt.ylim([0, 1])  # Reemplaza 'ymin' y 'ymax' con los valores deseados

plt.xlabel('bla bla bla')
plt.ylabel('bla bla bla')
plt.title('Titulo')
plt.show()

¿ qué podemos observar en esta gráfica?

< aquí sus comentarios >

**Material opcional**

In [ ]:
#Calculemos el coeficiente de correlacion
correlation = ml['Lead time'].corr(ml['q Defecto'])
print(f"Correlación entre Lead time y q Defecto: {correlation}")

In [ ]:
#Utilizaremos scikit-learn para ajustar un modelo de regresión lineal:

from sklearn.linear_model import LinearRegression

# Reshape the data
X = ml['Lead time'].values.reshape(-1, 1)  # Convertir a formato columna
y = ml['q Defecto'].values

# Crear y ajustar el modelo
model = LinearRegression()
model.fit(X, y)

# Coeficientes del modelo
slope = model.coef_[0]
intercept = model.intercept_
print(f"Pendiente: {slope}")
print(f"Intercepto: {intercept}")

In [ ]:
# Crear la gráfica de dispersión
plt.figure(figsize=(10, 6))
sns.scatterplot(x='Lead time', y='q Defecto', data=ml, color='r', label='Datos')

# Generar predicciones para la línea de tendencia
y_pred = model.predict(X)

# Graficar la línea de tendencia
plt.plot(ml['Lead time'], y_pred, color='blue', label='Línea de tendencia')

# Configurar etiquetas y título
plt.xlabel('Lead time')
plt.ylabel('q Defecto')
plt.title('Relación entre Lead time y q Defecto')
plt.legend()

# Mostrar la gráfica
plt.show()

### Gráfica Boxplot

In [ ]:
# Crear un Box Plot
plt.figure(figsize=(10, 6))

sns.boxplot(ml, x='WIP', y='Lead time', color='r')
# Definir los rangos de los ejes X e Y
#plt.ylim([0, 1])  # Reemplaza 'ymin' y 'ymax' con los valores deseados

plt.xlabel('Lead time')
plt.ylabel('WIP')
plt.title('Titulo')
plt.show()

¿qué ves en el Boxplot?, ¿cuáles son los elementos principales de est gráfico?

< Tu respuesta aquí >

**Bonus**

Investiga sobre un gráfico llamado Violinplot:

https://seaborn.pydata.org/generated/seaborn.violinplot.html

### Run chart

In [ ]:
# Filtrar el dataset para WIP = 20
ml_wip20 = ml.query('WIP == 20')
ml_wip6 = ml.query('WIP == 6')
ml_wip3 = ml.query('WIP == 3')

# Crear el runchart
plt.figure(figsize=(10, 6))
plt.plot(ml_wip20['Producto terminado'],ml_wip20['Lead time'], color='r', label='WIP=20')
plt.plot(ml_wip6['Producto terminado'],ml_wip6['Lead time'], color='g', label='WIP=6')
plt.plot(ml_wip3['Producto terminado'],ml_wip3['Lead time'], color='b', label='WIP=3')

plt.xlabel('Producto terminado')
plt.ylabel('Lead time')
plt.title('Run Chart para distintos escenarios de WIP Limit')
plt.legend()
plt.show()

¿qué podrías interpretar del análisis de esta gráfica?


## Opcional - grandes datasets

Leer dataset de salarios de Data developers:

In [ ]:
data = pd.read_csv("/kaggle/input/data-eng-salary-2024/Dataset salary 2024.csv")
data.head()

In [ ]:
#Calcula la media de Salario en dolares
#tip: usa Numpy 


In [ ]:
#crea un histograma del Salary


## Cierre


¿Tienes alguna pregunta o sugerencia? Déjame comentarios en https://www.linkedin.com/in/robertomoragad/ y haré todo lo posible para responder.

Más ejemplos:
* https://github.com/robertomoragad/Estadistica